# Required libraries

In [1]:
#Importing bunch of libraries.
!pip install memory_profiler
import os
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt 
from keras.models import Sequential,Model,load_model
from keras.layers import Conv2D,MaxPool2D,Dense,GlobalMaxPool2D,Dropout
import glob
from memory_profiler import memory_usage
import gc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Download openslr data

In [2]:
#Downloading the data
!wget http://www.openslr.org/resources/12/train-clean-100.tar.gz
!wget http://www.openslr.org/resources/12/test-clean.tar.gz

--2022-11-23 04:45:03--  http://www.openslr.org/resources/12/train-clean-100.tar.gz
Resolving www.openslr.org (www.openslr.org)... 46.101.158.64
Connecting to www.openslr.org (www.openslr.org)|46.101.158.64|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://us.openslr.org/resources/12/train-clean-100.tar.gz [following]
--2022-11-23 04:45:04--  https://us.openslr.org/resources/12/train-clean-100.tar.gz
Resolving us.openslr.org (us.openslr.org)... 46.101.158.64
Connecting to us.openslr.org (us.openslr.org)|46.101.158.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6387309499 (5.9G) [application/x-gzip]
Saving to: ‘train-clean-100.tar.gz’

train-clean-100.tar 100%[===================>]   5.95G  28.5MB/s    in 3m 36s  

2022-11-23 04:48:41 (28.1 MB/s) - ‘train-clean-100.tar.gz’ saved [6387309499/6387309499]

--2022-11-23 04:48:41--  http://www.openslr.org/resources/12/test-clean.tar.gz
Resolving www.openslr.org (www.openslr.or

In [3]:
!mkdir /content/train_data_wav/
!mkdir /content/test_data_wav/

In [4]:
!tar -xvf /content/test-clean.tar.gz -C /content/test_data_wav/
!tar -xvf /content/train-clean-100.tar.gz -C /content/train_data_wav/
# !rm /content/test-clean.tar.gz
# !rm /content/train-clean-100.tar.gz

Streaming output truncated to the last 5000 lines.
LibriSpeech/train-clean-100/2136/5143/2136-5143-0024.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0012.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0044.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0003.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0010.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0025.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0043.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0004.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0026.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0015.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0032.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0027.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0039.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0020.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0023.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0007.flac
LibriSpeech/train-cle

# Create a function for spectrogram

In [5]:
def create_spectrogram(filepath,save_path):
  plt.interactive(False)
  clip,sample_rate=librosa.load(filepath,sr=None)
  fig=plt.figure(figsize=[0.72,0.72])
  ax=fig.add_subplot(111)
  ax.axes.get_xaxis().set_visible(False)
  ax.axes.get_yaxis().set_visible(False)
  ax.set_frame_on(False)
  S=librosa.feature.melspectrogram(y=clip,sr=sample_rate)
  librosa.display.specshow(librosa.power_to_db(S,ref=np.max))
  fig.savefig(save_path,dpi=400,bbox_inches='tight',pad_inches=0)
  plt.close()
  fig.clf()
  plt.close(fig)
  plt.close('all')
  del filepath,save_path,clip,sample_rate,fig,ax,S

# Finding all files in a directory: Train data.

In [6]:
train_list=[]

path = '/content/train_data_wav/LibriSpeech/train-clean-100/'

files = []

for r, d, f in os.walk(path):
    # print(r, d, f)
    for flac_file in f:
        if '.flac' in flac_file:
            files.append(os.path.join(r, flac_file))
            

for f in files:
    train_list.append(f)

In [7]:
!ls /content/data_wav/LibriSpeech/train-clean-100/

ls: cannot access '/content/data_wav/LibriSpeech/train-clean-100/': No such file or directory


In [8]:
train_list[:3]

['/content/train_data_wav/LibriSpeech/train-clean-100/4859/22176/4859-22176-0008.flac',
 '/content/train_data_wav/LibriSpeech/train-clean-100/4859/22176/4859-22176-0001.flac',
 '/content/train_data_wav/LibriSpeech/train-clean-100/4859/22176/4859-22176-0004.flac']

#Finding all files in a directory: test data

In [9]:
test_list=[]

path = '/content/test_data_wav/LibriSpeech/test-clean/'

files = []

for r, d, f in os.walk(path):
    # print(r, d, f)
    for file in f:
        if '.flac' in file:
            files.append(os.path.join(r, file))

for f in files:
    test_list.append(f)

In [10]:
test_list[:3]

['/content/test_data_wav/LibriSpeech/test-clean/8463/294828/8463-294828-0006.flac',
 '/content/test_data_wav/LibriSpeech/test-clean/8463/294828/8463-294828-0012.flac',
 '/content/test_data_wav/LibriSpeech/test-clean/8463/294828/8463-294828-0036.flac']

In [11]:
!mkdir train_images
!mkdir test_images

# Creating spectrograms and storing to their respective directory

In [12]:
import memory_profiler as mem_profile

In [13]:
#Train_data
%load_ext memory_profiler

%memit   #Emit memory
for filepath in train_list:
  filename=filepath.split('/')[-1].split('.')[0]
  create_spectrogram(filepath,'/content/train_images/'+filename+'.jpg')
  print(f'/content/test_images/{filename}.jpg')

gc.collect()

#Test data
%memit   #Emit memory
for filepath in test_list:
  filename=filepath.split('/')[-1].split('.')[0]
  create_spectrogram(filepath,'/content/test_images/'+filename+'.jpg')
  print(f'/content/test_images/{filename}.jpg')

gc.collect()

Streaming output truncated to the last 5000 lines.
/content/test_images/5022-29405-0023.jpg
/content/test_images/5022-29405-0002.jpg
/content/test_images/5022-29405-0016.jpg
/content/test_images/5022-29405-0008.jpg
/content/test_images/5022-29405-0009.jpg
/content/test_images/5022-29405-0017.jpg
/content/test_images/5022-29405-0042.jpg
/content/test_images/5022-29405-0041.jpg
/content/test_images/5022-29405-0006.jpg
/content/test_images/5022-29405-0034.jpg
/content/test_images/5022-29405-0024.jpg
/content/test_images/5022-29405-0038.jpg
/content/test_images/5022-29405-0022.jpg
/content/test_images/5022-29405-0032.jpg
/content/test_images/5022-29405-0029.jpg
/content/test_images/5022-29405-0020.jpg
/content/test_images/5022-29405-0011.jpg
/content/test_images/5022-29405-0015.jpg
/content/test_images/5022-29411-0029.jpg
/content/test_images/5022-29411-0001.jpg
/content/test_images/5022-29411-0070.jpg
/content/test_images/5022-29411-0060.jpg
/content/test_images/5022-29411-0056.jpg
/conte

6637

# Building the batch generator

In [14]:
import cv2
def load_img(path):
  img=cv2.imread(path)
  img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  img=cv2.resize(img,(150,150))
  return img

def generate_idxs(X):
  idxs1=np.random.randint(0,len(X),1)[0]
  return idxs1

def same_label_index(X):

  while True:
    idx1=generate_idxs(X)
    idx2=generate_idxs(X)
    file1_dir=X[idx1]
    file2_dir=X[idx2]
    lb1=file1_dir.split("/")[-1].split("-")[0]
    lb2=file2_dir.split("/")[-1].split("-")[0]
    if lb1==lb2:
      break

  return idx1,idx2

def different_label_index(X):
  while True:
    idx1=generate_idxs(X)
    idx2=generate_idxs(X)
    file1_dir=X[idx1]
    file2_dir=X[idx2]
    lb1=file1_dir.split("/")[-1].split("-")[0]
    lb2=file2_dir.split("/")[-1].split("-")[0]
    if lb1!=lb2:
      break

  return idx1,idx2

def batch_generator(X,batch_size):
  while True:
    data=[np.zeros((batch_size,150,150,3)) for i in range(2)]
    tar=[np.zeros(batch_size,)]

    #Generating same pairs.
    for i in range(0,batch_size//2):
      idx1,idx2=same_label_index(X)
      img1=load_img(X[idx1])
      img1=img1/255
      img2=load_img(X[idx2])
      img2=img2/255

      data[0][i,:,:,:]=img1
      data[1][i,:,:,:]=img2
      tar[0][i]=1

    #Generating different pairs.
    for k in range(batch_size//2,batch_size):
      idx1,idx2=different_label_index(X)
      img1=load_img(X[idx1])
      img1=img1/255
      img2=load_img(X[idx2])
      img2=img2/255

      data[0][k,:,:,:]=img1
      data[1][k,:,:,:]=img2
      tar[0][k]=0
    np.delete(data[0],np.where(~data[0].any(axis=1))[0], axis=0)
    np.delete(data[1],np.where(~data[1].any(axis=1))[0], axis=0)
    yield data,tar


In [15]:
%matplotlib inline
img_test=cv2.imread('/content/train_images/587-54108-0035.jpg')

# img_test=cv2.cvtColor(img_test,cv2.COLOR_BGR2RGB)
# plt.imshow(img_test)
print(img_test)

[[[ 11   3   3]
  [  9   4   3]
  [  3   0   2]
  ...
  [146  57 127]
  [ 94   4  69]
  [109  18  67]]

 [[ 12   2   2]
  [ 10   2   2]
  [  2   0   0]
  ...
  [143  56 130]
  [ 91   2  71]
  [106  14  65]]

 [[ 23   8   6]
  [ 19   7   7]
  [  7   2   3]
  ...
  [142  60 142]
  [ 89   2  76]
  [113  18  75]]

 ...

 [[139  50 160]
  [126  46 135]
  [ 81  16  62]
  ...
  [119  67 205]
  [111  55 190]
  [117  56 190]]

 [[142  50 163]
  [124  39 131]
  [ 84  15  65]
  ...
  [118  72 215]
  [104  55 195]
  [111  55 192]]

 [[136  44 157]
  [128  41 131]
  [ 91  17  65]
  ...
  [118  81 227]
  [106  62 205]
  [113  59 196]]]


# Getting training image list

In [16]:
X_train=[]
temp_file_list_= os.listdir('/content/train_images/')
for filename in temp_file_list_:
  X_train.append(str('/content/train_images/'+filename))

# Getting testing image list

In [17]:
X_test=[]
temp_file_list_= os.listdir('/content/test_images/')
for filename in temp_file_list_:
  X_test.append(str('/content/test_images/'+filename))

# Building Model

In [18]:
from keras.layers import Lambda,Input
import keras.backend as K
from keras.callbacks import ModelCheckpoint,EarlyStopping

def cosine_distance(vests):
    x, y = vests
    x = K.l2_normalize(x, axis=-1)
    y = K.l2_normalize(y, axis=-1)
    return -K.mean(x * y, axis=-1, keepdims=True)

def get_encoder(input_size):
  model=Sequential()
  model.add(Conv2D(32,(3,3),input_shape=(150,150,3),activation='relu'))
  model.add(Conv2D(64,(3,3),activation='relu'))
  model.add(MaxPool2D(2,2))
  model.add(Dropout(0.2))

  model.add(Conv2D(64,(3,3),activation='relu'))
  model.add(Conv2D(64,(3,3),activation='relu'))
  model.add(MaxPool2D(2,2))
  model.add(Dropout(0.2))

  model.add(Conv2D(128,(3,3),activation='relu'))
  model.add(Conv2D(128,(3,3),activation='relu'))
  model.add(MaxPool2D(2,2))
  model.add(Dropout(0.2))

  model.add(GlobalMaxPool2D())

  return model

# Creating siamens neural network

In [19]:
def get_siamese_network(encoder,input_size):
  input1=Input(input_size)
  input2=Input(input_size)

  encoder_l=encoder(input1)
  encoder_r=encoder(input2)
  
  L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
  L1_distance = L1_layer([encoder_l, encoder_r])

  output=Dense(1,activation='sigmoid')(L1_distance)
  siam_model=Model(inputs=[input1,input2],outputs=output)
  return siam_model

encoder=get_encoder((150,150,3))
siamese_net=get_siamese_network(encoder,(150,150,3))
siamese_net.compile(loss='binary_crossentropy',optimizer='adam')


# Avoid overfitting through early stopping

In [20]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, min_delta=0.0001) 
mc = ModelCheckpoint('SpeakerID_best.hdf5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [21]:
history=siamese_net.fit(batch_generator(X_train,10),steps_per_epoch=150,epochs=30,validation_data=batch_generator(X_test,10),
                            validation_steps=20,callbacks=[es,mc],shuffle=True,use_multiprocessing=True)

Epoch 1/30
150/150 [==============================] - ETA: 0s - loss: 0.6887
Epoch 1: val_loss improved from inf to 0.67497, saving model to SpeakerID_best.hdf5
150/150 [==============================] - 26s 105ms/step - loss: 0.6887 - val_loss: 0.6750
Epoch 2/30
150/150 [==============================] - ETA: 0s - loss: 0.6610
Epoch 2: val_loss did not improve from 0.67497
150/150 [==============================] - 16s 109ms/step - loss: 0.6610 - val_loss: 0.6817
Epoch 3/30
150/150 [==============================] - ETA: 0s - loss: 0.6657
Epoch 3: val_loss did not improve from 0.67497
150/150 [==============================] - 16s 106ms/step - loss: 0.6657 - val_loss: 0.6828
Epoch 4/30
150/150 [==============================] - ETA: 0s - loss: 0.6408
Epoch 4: val_loss improved from 0.67497 to 0.66333, saving model to SpeakerID_best.hdf5
150/150 [==============================] - 16s 106ms/step - loss: 0.6408 - val_loss: 0.6633
Epoch 5/30
150/150 [==============================] - ETA:

# Different pair+ Same pair

In [25]:

idx1,idx2=same_label_index(X_test)
X1=[np.zeros((1,150,150,3)) for i in range(2)]

img1=load_img(X_test[idx1])
img1=np.array(img1)
img1=img1/255
img2=load_img(X_test[idx2])
img2=np.array(img2)
img2=img2/255

X1[0][0,:,:,:]=img1
X1[1][0,:,:,:]=img2


idx1,idx2=different_label_index(X_test)
X2=[np.zeros((1,150,150,3)) for i in range(2)]

img1=load_img(X_test[idx1])
img1=np.array(img1)
img1=img1/255
img2=load_img(X_test[idx2])
img2=np.array(img2)
img2=img2/255


X2[0][0,:,:,:]=img1
X2[1][0,:,:,:]=img2

model=load_model('/content/SpeakerID_best.hdf5')
print(model.predict(X1))
print(model.predict(X2))

1/1 [==============================] - 0s 123ms/step
[[0.5541446]]
1/1 [==============================] - 0s 16ms/step
[[0.6856794]]


In [26]:
from google.colab import files
files.download('/content/SpeakerID_best.hdf5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>